### Data Preparation

In [1]:
NUM_OF_LABELS = 9
BATCH_SIZE = 32

In [2]:
import tensorflow as tf
from Load_Data import load_image_dataset_from_directory

# filepaths
b_dir = '/datasets/small_datasets/coffee'
#b_dir = '/datasets/rock_paper_scissor_original'
tr_dir = '/train'
val_dir = '/train'
ts_dir = '/test'

# loading data
train_dataset, valid_dataset, test_dataset = load_image_dataset_from_directory(b_dir, tr_dir, val_dir, ts_dir)

c:\Users\Programming\anaconda3\envs\c_project\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Found 450 files belonging to 9 classes.
Using 360 files for training.
Found 450 files belonging to 9 classes.
Using 90 files for validation.
Found 450 files belonging to 9 classes.


### Model(VGG19+SPP)

* Spatial_Pyramid_Pooling(https://arxiv.org/abs/1406.4729) code is provided by yhenon(https://github.com/yhenon/keras-spp)

In [3]:
import os, sys
sys.path.append(os.getcwd()+'/sppnet')
from SpatialPyramidPooling import SpatialPyramidPooling

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Normalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense



def make_model():
    model = Sequential()

    #model.add(Normalization())
    
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(strides=(2,2)))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(strides=(2,2)))

    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(strides=(2,2)))

    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(strides=(2,2)))

    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(SpatialPyramidPooling([1,2,4]))
    #model.add(MaxPooling2D(strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(units=4096, activation='relu', ))
    model.add(Dense(units=4096, activation='relu', input_shape=(None,4096) ))
    model.add(Dense(units=NUM_OF_LABELS, activation='softmax', input_shape=(None,4096) ))
    
    return model

In [5]:
def compile_model(model):
    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy']
                )

### Model Training

In [6]:
import numpy as np

model = make_model()
compile_model(model)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="VGG19",
        save_best_only=True,
        monitor="val_loss"
    )
]

history = model.fit(
    x = train_dataset,
    epochs=100,
    validation_data=valid_dataset,
    callbacks=callbacks,
    use_multiprocessing=True,
    )
model.summary()

Epoch 1/100
12/12 [==============================] - ETA: 0s - loss: 2198.9917 - accuracy: 0.1167

INFO:tensorflow:Assets written to: VGG19\assets


INFO:tensorflow:Assets written to: VGG19\assets


12/12 [==============================] - 33s 2s/step - loss: 2198.9917 - accuracy: 0.1167 - val_loss: 2.1941 - val_accuracy: 0.1556
Epoch 2/100
12/12 [==============================] - 7s 574ms/step - loss: 2.2555 - accuracy: 0.0944 - val_loss: 2.2010 - val_accuracy: 0.1000
Epoch 3/100
12/12 [==============================] - 7s 563ms/step - loss: 2.2018 - accuracy: 0.1139 - val_loss: 2.1992 - val_accuracy: 0.1000
Epoch 4/100
12/12 [==============================] - 7s 565ms/step - loss: 2.2017 - accuracy: 0.1139 - val_loss: 2.2000 - val_accuracy: 0.1000
Epoch 5/100
12/12 [==============================] - 7s 566ms/step - loss: 2.1976 - accuracy: 0.0972 - val_loss: 2.1996 - val_accuracy: 0.1222
Epoch 6/100
12/12 [==============================] - 7s 565ms/step - loss: 2.2046 - accuracy: 0.1389 - val_loss: 2.1997 - val_accuracy: 0.0778
Epoch 7/100
12/12 [==============================] - 7s 565ms/step - loss: 2.1973 - accuracy: 0.1194 - val_loss: 2.2008 - val_accuracy: 0.0778
Epoch 8/10

+ WARNING:absl:Found untraced functions such as ... while saving. These functions will not be directly callable after loading.
: ModelCheckpoint를 사용하는 과정에서 발견되는 오류. tf.saved_model.save 대신 model.save를 사용할 것을 권장하기 위해 출력시키는 경고라는 의견이 있음.
(https://github.com/tensorflow/tensorflow/issues/47479#issuecomment-815314034)

### Prediction Accuracy

In [7]:
model = keras.models.load_model("VGG19")
print(f"테스트 정확도: {model.evaluate(test_dataset)[1]:.3f}")

15/15 [==============================] - 3s 181ms/step - loss: 2.1972 - accuracy: 0.1156
테스트 정확도: 0.116


In [8]:
y_predict = model.predict(test_dataset)
predicted_labels = y_predict.argmax(axis=-1)
print(f"테스트 정확도: {model.evaluate(test_dataset)[1]}")
print(f"예측 라벨: {predicted_labels}")
print(f"예측 정도: {model.predict(test_dataset)}")
print(f"{len(predicted_labels)} data are predicted.")

15/15 [==============================] - 2s 144ms/step - loss: 2.1972 - accuracy: 0.1156
테스트 정확도: 0.11555555462837219
예측 라벨: [6 8 6 8 6 8 6 8 6 6 6 8 8 8 8 6 6 8 8 8 8 6 8 8 8 8 8 8 6 8 6 8 8 8 8 6 6
 8 8 8 8 8 8 6 8 8 8 8 6 6 8 8 8 6 8 8 6 8 8 8 8 8 8 8 6 8 8 6 8 6 6 8 8 8
 8 8 8 6 8 8 8 8 6 6 8 8 6 8 8 8 8 6 6 8 6 8 8 8 8 8 8 8 6 8 8 8 8 8 8 6 6
 8 6 6 8 8 6 8 8 8 8 6 8 8 8 8 6 8 8 8 8 8 8 6 6 6 6 8 8 8 8 8 8 6 8 6 8 8
 8 8 8 6 6 6 8 8 8 8 8 6 6 8 8 6 8 6 8 6 8 8 8 8 8 8 6 8 8 8 8 6 8 8 8 8 8
 6 8 6 8 6 8 6 8 8 8 8 8 8 8 6 6 8 8 8 8 6 8 8 8 8 6 8 6 8 8 8 8 8 6 8 6 6
 6 8 8 8 6 6 6 8 8 6 6 8 8 8 8 8 8 6 8 8 8 6 6 6 8 8 8 6 6 8 6 8 8 6 8 6 8
 6 8 6 8 6 8 8 8 8 8 8 8 8 8 6 8 6 8 8 6 8 6 8 8 8 8 6 8 8 6 8 8 8 8 8 6 8
 6 8 8 8 8 8 8 6 8 8 6 8 6 8 6 8 6 6 8 8 8 8 8 8 8 8 8 8 6 8 8 8 6 6 8 8 8
 8 8 8 8 8 6 8 8 6 8 6 8 8 8 8 8 8 8 6 8 6 8 8 6 8 8 8 8 6 8 8 6 6 6 6 6 6
 6 6 6 8 6 8 8 8 8 6 6 8 6 6 6 6 6 8 6 6 8 8 8 8 8 8 6 8 6 8 8 8 8 8 8 8 8
 8 8 6 8 6 8 8 6 6 8 8 8 6 8 8 8 8 8 8 8 8 8 8 8 6